In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from datetime import timedelta
from long_covid.colors import flatuicolors
from long_covid import styling
import datetime 
import matplotlib.dates as mdates

In [ ]:
COHORTS = pd.read_feather('../data/03_derived/user_cohorts.feather')
TESTS = pd.read_feather('../data/01_raw/tests.feather')
WEEKLY_DEVIATIONS = pd.read_feather('../data/03_derived/weekly_vital_deviations_per_user.feather')

In [ ]:
def monthly_reports():
    
    f, ax = plt.subplots(figsize=(4.5,3))

    unvaccinated_users = COHORTS.user_id[COHORTS['unvaccinated']].values 
    vaccinated_users = COHORTS.user_id[COHORTS['vaccinated']].values 
    
    users = WEEKLY_DEVIATIONS[WEEKLY_DEVIATIONS.userid.isin(unvaccinated_users)].userid.unique()
    tests = TESTS[TESTS.user_id.isin(users)]
    tests.set_index('test_date', inplace=True)
    unvaccinated_users = tests.resample('1M').count()
    
    users = WEEKLY_DEVIATIONS[WEEKLY_DEVIATIONS.userid.isin(vaccinated_users)].userid.unique()
    tests = TESTS[TESTS.user_id.isin(users)]
    tests.set_index('test_date', inplace=True)
    vaccinated_users = tests.resample('1M').count()
        
    plot_data = pd.merge(vaccinated_users, unvaccinated_users, how='outer', on='test_date')
    plot_data.sort_index(inplace=True)
    plot_data = plot_data[2:]
    
    ax.bar(plot_data.index - timedelta(days=30), plot_data.user_id_x, width=20, color=flatuicolors.amethyst, label='Vaccinated')
    ax.bar(plot_data.index - timedelta(days=30), plot_data.user_id_y.fillna(0), width=20, bottom=plot_data.user_id_x.fillna(0), color=flatuicolors.pomegranate, label='Unvaccinated')
    ax.legend(loc='upper left')

    ax.set_ylabel('Number of self-reported\npositive PCR-tests')
    
    styling.hide_and_move_axis(ax)
    ax.semilogy()
    ax.set_ylim(0.5, 2E3)
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
    
    plt.tight_layout()
    plt.savefig('../output/si_figure1_pcr_counts.jpg', dpi=400)

In [ ]:
monthly_reports()